# Milestone 1 Playground -- Ricky's Notebook
---
Cell for importing packages:

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
# !{sys.executable} -m pip install pandas

  Using cached pandas-1.0.1-cp38-cp38-macosx_10_9_x86_64.whl (9.9 MB)
  Using cached pytz-2019.3-py2.py3-none-any.whl (509 kB)


Imports cell:

In [1]:
# 441975, l.teixeira@wustl.edu, Teixeira, Lucas
# 443896, rickynoll@wustl.edu, Noll, Ricky
# XXXXXX, XXXXX@wustl.edu, Kowsari, Daria

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import linear_model
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

---

In [2]:
# Read Input files from current directory
train = pd.read_csv(os.path.join(os.getcwd(),'train.csv'))
X = train.loc[:, 'ID':'Soil_Type']
y = train.loc[:, 'Horizontal_Distance_To_Fire_Points'::]
test = pd.read_csv(os.path.join(os.getcwd(),'test.csv'))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

print("X_train: ", X_train.shape)
print("X_val  : ", X_val.shape)
print("y_train: ", y_train.shape)
print("y_val  : ", y_val.shape)
print("test   : ", test.shape)
# Any results you write to the current directory are saved as output.

X_train:  (4983, 11)
X_val  :  (2455, 11)
y_train:  (4983, 1)
y_val  :  (2455, 1)
test   :  (11157, 11)


In [3]:
X_train[X_train.columns[1:]]

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Soil_Type
2492,3318,31,12,90,5,2101,216,213,130,8771
1654,3268,58,9,228,29,934,227,222,129,7756
6452,3161,120,9,426,32,488,236,233,128,7202
6816,3167,328,4,60,3,566,209,234,164,7202
2323,3382,273,28,547,176,700,135,234,232,7757
...,...,...,...,...,...,...,...,...,...,...
5191,3371,36,9,190,39,1550,220,221,136,8776
5226,3463,141,16,120,14,2081,242,233,115,8772
5390,3301,291,14,703,102,767,181,237,196,7201
860,3184,344,12,182,6,1103,195,220,164,7700


In [4]:
y_train.squeeze()

2492    2112
1654     849
6452    5706
6816    1295
2323     942
        ... 
5191    1959
5226    3087
5390    1024
860      750
7270    1065
Name: Horizontal_Distance_To_Fire_Points, Length: 4983, dtype: int64

---
## Linear Regression
Now that we have Train the simplest model possible.

__Note:__ You have to cut off the first column because the `id's` are not a real feature.

In [5]:
model = linear_model.LinearRegression()
model.fit(X_train[X_train.columns[1:]], y_train.squeeze())

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [6]:
print(f"model.coef_: {model.coef_}\n")
print(f"model.intercept_: {model.intercept_}\n")

y_pred = model.predict(X_val[X_val.columns[1:]])

print("--- LinearReg Stats ---")
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f"MSE : {mse}")
print(f"RMSE: {rmse}")
print(f"MAE : {mae}")
print(f"R^2  : {r2}")

model.coef_: [  1.43494619  -3.59996913  79.0550337    0.1140299   -5.95011658
   0.37881371  40.46475194 -15.87906337  25.82618215  -0.09271265]

model.intercept_: -11032.046887026792

--- LinearReg Stats ---
MSE : 1221421.671899665
RMSE: 1105.1794749721264
MAE : 855.36387287251
R^2  : 0.340525252981012


Retrain with __ALL__ data. Write predictions out to file:

In [7]:
model.fit(X[X.columns[1:]], y.squeeze())
test_preds = model.predict(test[test.columns[1:]])
submission_output = pd.DataFrame(data={'ID': test.ID, 'Horizontal_Distance_To_Fire_Points': test_preds})
submission_output.to_csv(os.path.join(os.getcwd(),'result_linreg_ricky.csv'), index=False)
submission_output

,ID,Horizontal_Distance_To_Fire_Points
0,49513387,3597.645929
1,154482561,3900.906467
2,170918549,3224.845915
3,483460773,1090.325349
4,3421968690,1153.521955
...,...,...
11152,9996758076316,2955.786024
11153,9997790496155,1765.567301
11154,9998021658144,2266.622129
11155,9998459291425,1001.881199


---
## LinearSVR

In [11]:
from sklearn.svm import LinearSVR
# from sklearn.datasets import make_regression
# X, y = make_regression(n_features=4, random_state=0)

tuned_param = {'C':[0.1, 1., 5., 10., 20., 50.], 'epsilon':[0, 0.1, 1., 5.]}
svc = LinearSVR(dual=False, loss='squared_epsilon_insensitive')
grid_search = GridSearchCV(svc, tuned_param, scoring='r2')
grid_search.fit(X_train[X_train.columns[1:]], y_train.squeeze())

regr = grid_search.best_estimator_
regr.fit(X_train[X_train.columns[1:]], y_train.squeeze())

LinearSVR(C=20.0, dual=False, epsilon=1.0, fit_intercept=True,
          intercept_scaling=1.0, loss='squared_epsilon_insensitive',
          max_iter=1000, random_state=None, tol=0.0001, verbose=0)

In [12]:
print(f"regr.coef_: {regr.coef_}\n")
print(f"regr.intercept_: {regr.intercept_}\n")

y_pred = regr.predict(X_val[X_val.columns[1:]])

print("--- LinearSVR Stats ---")
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f"MSE : {mse}")
print(f"RMSE: {rmse}")
print(f"MAE : {mae}")
print(f"R^2 : {r2}")

regr.coef_: [  0.61093045  -3.74273161  37.24441493   0.22565299  -5.27749764
   0.46660108  -6.9319      17.78607579 -12.16210886  -0.08428745]

regr.intercept_: [-0.05796345]

--- LinearSVR Stats ---
MSE : 1231154.3044311525
RMSE: 1109.5739292319158
MAE : 865.9004969375745
R^2 : 0.3352703721121071


Retrain with __ALL__ data. Write predictions out to file:

In [13]:
regr.fit(X[X.columns[1:]], y.squeeze())
test_preds = regr.predict(test[test.columns[1:]])
submission_output = pd.DataFrame(data={'ID': test.ID, 'Horizontal_Distance_To_Fire_Points': test_preds})
submission_output.to_csv(os.path.join(os.getcwd(),'result_svr_ricky.csv'), index=False)
submission_output

,ID,Horizontal_Distance_To_Fire_Points
0,49513387,3569.331001
1,154482561,3809.234219
2,170918549,2832.750566
3,483460773,1057.524473
4,3421968690,1408.485970
...,...,...
11152,9996758076316,2513.809786
11153,9997790496155,1635.563281
11154,9998021658144,2429.141097
11155,9998459291425,1100.294008


---
## SGDRegressor

In [10]:
sgd_regr = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)
sgd_regr.fit(X_train[X_train.columns[1:]], y_train.squeeze())

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [11]:
print(f"regr.coef_: {sgd_regr.coef_}\n")
print(f"regr.intercept_: {sgd_regr.intercept_}\n")

y_pred = sgd_regr.predict(X_val[X_val.columns[1:]])

print("--- SGDRegressor Stats ---")
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f"MSE : {mse}")
print(f"RMSE: {rmse}")
print(f"MAE : {mae}")
print(f"R^2  : {r2}")

regr.coef_: [-1.68717326e+12  1.16398144e+12 -4.19012002e+12 -8.82560641e+11
 -5.07427188e+11 -1.02120955e+12  3.64339449e+12  7.91068526e+10
 -1.32852566e+11 -1.09056734e+12]

regr.intercept_: [-7.18763167e+08]

--- SGDRegressor Stats ---
MSE : 2.003060121343028e+32
RMSE: 1.4152950651164682e+16
MAE : 1.4101398334034398e+16
R^2  : -1.0815000234374661e+26
